In [88]:
# visualize the data
import numpy as np
import pandas as pd
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
# Load dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
data = read_csv(url, names=names)

In [89]:
def sigmoid(S):
  """
  S: an numpy array
  return sigmoid function of each element of S
  """
  return 1/(1 + np.exp(-S))

In [90]:
def prob(w, X):
  """
  X: a 2d numpy array of shape (N, d). N datatpoint, each with size d
  w: a 1d numpy array of shape (d)
  """
  return sigmoid(X.dot(w))


In [91]:
def loss(w, X, y, lam):
  """
  X, w as in prob
  y: a 1d numpy array of shape (N). Each elem = 0 or 1
  """
  z = prob(w, X)
  return -np.mean(y*np.log(z) + (1-y)*np.log(1-z)) + 0.5*lam/X.shape[0]*np.sum(w*w)

In [92]:
def loss_gradient(w, X, y, lam):
  """
  X, w as in prob
  y: a 1d numpy array of shape (N). Each elem = 0 or 1
  """
  z = sigmoid(X.dot(w))
  return (z - y).dot(X) + lam*w

In [93]:
def logistic_regression(w_init, X, y, lam = 0.001, lr = 0.1, nepoches = 2000):
  # lam - reg paramether, lr - learning rate, nepoches - number of epoches
  N, d = X.shape[0], X.shape[1]
  w = w_init
  ep = 0
  while ep < nepoches:
    ep += 1
    mix_ids = np.random.permutation(N)
    for i in mix_ids:
      xi = X[i]
      yi = y[i]
      zi = sigmoid(xi.dot(w))
      w = w - lr*((zi - yi)*xi + lam*w)
    if np.linalg.norm(loss_gradient(w, X, y, lam) < 1e-6):
      break
  return w

In [94]:
from sklearn.model_selection import train_test_split
# Split-out validation dataset
X = data.loc[:, :'petal-width']
y = data['class']
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [95]:
#train data for model 1
#Split data for class 1 vs 2
X1 = X_train.loc[data['class'] != 'Iris-virginica']
y1_class = Y_train.loc[data['class'] != 'Iris-virginica']
y1 = np.zeros(X1.shape[0])
itr = 0
for index, row in y1_class.items():
  if row == 'Iris-setosa':
    y1[itr] = 1
  else:
    y1[itr] = 0
  itr += 1
Xbar1 = np.concatenate((X1, np.ones((X1.shape[0], 1))), axis = 1)
w_init1 = np.zeros(Xbar1.shape[1])
lam = 0.0001
w1 = logistic_regression(w_init1, Xbar1, y1, lam)
w1

array([ 0.35362743,  1.31478746, -2.01499766, -0.83378213,  0.21146232])

In [96]:
#train data for model 2
#Split data for class 1 vs 3
X2 = X_train.loc[data['class'] != 'Iris-versicolor']
y2_class = Y_train.loc[data['class'] != 'Iris-versicolor']
N = X2.shape[0]
y2 = np.zeros(N)
itr = 0
for index, row in y2_class.items():
  if row == 'Iris-setosa':
    y2[itr] = 1
  else:
    y2[itr] = 0
  itr += 1
Xbar2 = np.concatenate((X2, np.ones((N, 1))), axis = 1)
w_init2 = np.random.randn(Xbar2.shape[1])
lam = 0.0001
w2 = logistic_regression(w_init2, Xbar2, y2, lam)
w2

array([ 0.30486754,  0.90308284, -0.95021996, -1.71606452,  0.44676796])

In [97]:
#train data for model 3
#Split data for class 2 vs 3
X3 = X_train.loc[data['class'] != 'Iris-setosa']
y3_class = Y_train.loc[data['class'] != 'Iris-setosa']
N = X3.shape[0]
y3 = np.zeros(N)
itr = 0
for index, row in y3_class.items():
  if row == 'Iris-versicolor':
    y3[itr] = 1
  else:
    y3[itr] = 0
  itr += 1
Xbar3 = np.concatenate((X3, np.ones((N, 1))), axis = 1)
w_init3 = np.random.randn(Xbar3.shape[1])
lam = 0.0001
w3 = logistic_regression(w_init3, Xbar3, y3, lam)
w3

array([ 1.57977371,  1.53418436, -3.23746717, -1.26728172,  0.33297099])

In [98]:
def prediction(X, w1, w2, w3):
  N = X.shape[0]
  prob1 = prob(w1, X)
  prob2 = prob(w2, X)
  prob3 = prob(w3, X)
  pred = []
  for i in range(N):
    c1 = c2 = c3 = 0
    if prob1[i] > 0.5: c1 += 1
    else: c2 += 1
    if prob2[i] > 0.5: c1 += 1
    else: c3 += 1
    if prob3[i] > 0.5: c2 += 1
    else: c3 += 1
    mx = max(c1, c2, c3)
    if c1 == mx: pred.append( 'Iris-setosa') 
    elif c2 == mx: pred.append( 'Iris-versicolor')
    else: pred.append('Iris-virginica')
  return pred

In [99]:
from sklearn.metrics import accuracy_score
Xbar_test = np.concatenate((X_test, np.ones((X_test.shape[0], 1))), axis = 1)
y_pred = prediction(Xbar_test, w1, w2, w3)
print(accuracy_score(Y_test, y_pred))
test = pd.DataFrame([[1, 4.3, 3.0, 1.1, 0.1]])
print(prediction(test, w1, w2, w3))

0.7333333333333333
['Iris-virginica']


In [100]:
from sklearn.linear_model import LogisticRegression
logic = LogisticRegression(random_state = 0).fit(X_train, Y_train)
y_sk = logic.predict(X_test)
test = pd.DataFrame([[4.8, 3.5, 1.6, 0.7]])
print(logic.predict(test))
print(accuracy_score(Y_test, y_sk))

['Iris-setosa']
1.0


In [101]:
#For prediction of a single sample
test = pd.DataFrame([[1, 4.3, 3.0, 1.1, 0.1]])
print(prediction(test, w1, w2, w3))

['Iris-virginica']
